In [1]:
import pickle
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error as mse
from torch.utils.data import DataLoader
from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np

from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
with open('train_data.pkl', 'rb') as f:
    train_data = pickle.load(f)

with open('val_data.pkl', 'rb') as f:
    val_data = pickle.load(f)

with open('test_data.pkl', 'rb') as f:
    test_data = pickle.load(f)

In [63]:
N_COMPONENTS = 3
INPUT_STEP = 10
OUTPUT_STEP = 1
BATCH_SIZE = 8

## PCA

### Try with multiple frame

In [5]:
pca = PCA(n_components=N_COMPONENTS)

In [6]:
pca.fit(train_data)

PCA(n_components=3)

In [7]:
pca.components_

array([[ 0.22228766,  0.14876979,  0.10380687,  0.08584069,  0.07661014,
         0.07090557,  0.06700337,  0.0643168 ,  0.06277027,  0.06241932,
         0.06316392,  0.06451598,  0.06605107,  0.06751989,  0.06881513,
         0.06991322,  0.07082615,  0.07157469,  0.07218154,  0.07266664,
         0.07304902,  0.07334518,  0.07357087,  0.07373892,  0.07386177,
         0.07394961,  0.07401102,  0.07405275,  0.07408023,  0.07409766,
         0.07410806,  0.07411363,  0.07411634,  0.07411711,  0.07411706,
         0.07411669,  0.07411649,  0.07411653,  0.07411701,  0.07411797,
         0.07411934,  0.07412097,  0.07412291,  0.07412504,  0.07412734,
         0.07412976,  0.07413223,  0.07413475,  0.07413721,  0.07413968,
         0.07414198,  0.0741442 ,  0.07414646,  0.07414856,  0.07415058,
         0.07415237,  0.07415421,  0.07415588,  0.07415759,  0.07415919,
         0.07416074,  0.07416213,  0.07416352,  0.07416479,  0.07416597,
         0.07416706,  0.07416813,  0.07416912,  0.0

In [25]:
pca.explained_variance_

array([1.09692232e+06, 2.74428237e+00, 1.57192404e+00])

## Reconstructing the original data
Since we have the eigenvalues and eigenvectors of the covariance matrix of the pressure data (PCA), we can reconstruct the original data.

In [44]:
transformed_data = pca.transform(train_data)

In [45]:
reconstructed_sklearn = pca.inverse_transform(transformed_data)

In [46]:
mse(reconstructed_sklearn, train_data, squared=False)

0.012500153987806075

## Create dataset

In [ ]:
class HEDataset(Dataset):
    def __init__(timeframes):
        # standardizing data
        print(timeframes.mean(0))
        self.std = timeframes.std(0)
        transformed_data = timeframes/std
        self.srcs = transformed_data.unfold(0, INPUT_STEP, 1)
        self.tgts = torch.unsqueeze(transformed_data[INPUT_STEP:], 2)
        step = 10020
        assert torch.equal(self.srcs[step, :, -1].detach().unsqueeze(1), self.tgts[step-1, :, :].detach())
        print(self.srcs.shape, self.tgts.shape)

    def __getitem__(self, idx):
        src = self.srcs[idx, :, :]
        tgt = self.tgts[idx, :, :]
        return src, tgt
    
    def __len__(self):
        return self.srcs.shape(0)

In [47]:
transformed_data = torch.Tensor(transformed_data).to(device)

In [48]:
transformed_data.shape

torch.Size([984921, 3])

In [49]:
# Training
# 1 2 3 -> 4
# 2 3 4 -> 5
# Testing
# 3 4 5 -> (6) todo mse of this step
# 4 5 (6) -> (7) todo mse of this step
# 5 (6) (7) -> (8) todo mse of this step
# todo 5 is seen, so performance may be benefited from this

In [50]:
training_data = HEDataset

tensor([0., 0., 0.], device='cuda:0')


torch.Size([984912, 3, 10])

In [52]:
tgts = transformed_data[INPUT_STEP:]

torch.Size([984911, 3, 1])

In [70]:
srcs[1, :, :], tgts[1, :, :]

(tensor([[-0.0365,  1.5777, -1.1298,  0.8703, -0.3738, -0.8268,  0.9902,  0.0297,
          -1.5376,  0.3096],
         [-0.3521, -4.5794, -0.2453,  1.4905, -0.2080, -0.4197,  1.1899,  0.1261,
           0.2413, -0.1793],
         [-0.4027, -0.4468,  0.0340, -0.1474, -0.1838, -0.0961, -0.3438, -0.2225,
           0.4665, -0.4017]], device='cuda:0'),
 tensor([[ 1.2723],
         [ 0.6815],
         [-0.5560]], device='cuda:0'))

In [66]:
dataloader = DataLoader(srcs[:5000,:,:], batch_size=BATCH_SIZE, shuffle=False)

In [67]:
def train(model):
    model.train()
    loss = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-5)
    for epoch in range(3):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, src in enumerate(tqdm(dataloader)):
            import pdb; pdb.set_trace()
            optimizer.zero_grad()
            tgt = torch.unsqueeze(tgts[i], dim=0)
            outputs, _ = model(src)
            l = loss(outputs, tgt)
            l.backward()
            optimizer.step()
            # print statistics
            running_loss += l
        print(f'[{epoch + 1}] loss: {mean(running_loss)}')
    return model  

### LSTM

In [68]:
lstm = nn.LSTM(INPUT_STEP, OUTPUT_STEP, 8, batch_first=True).to(device)
lstm = train(lstm)

  0%|          | 0/625 [00:00<?, ?it/s]

> /tmp/ipykernel_20347/3655936383.py(9)train()
      7         for i, src in enumerate(tqdm(dataloader)):
      8             import pdb; pdb.set_trace()
----> 9             optimizer.zero_grad()
     10             tgt = torch.unsqueeze(tgts[i], dim=0)
     11             outputs, _ = model(src)



ipdb>  i


0


ipdb>  src


tensor([[[-1.1033, -0.0365,  1.5777, -1.1298,  0.8703, -0.3738, -0.8268,
           0.9902,  0.0297, -1.5376],
         [-0.2368, -0.3521, -4.5794, -0.2453,  1.4905, -0.2080, -0.4197,
           1.1899,  0.1261,  0.2413],
         [ 0.0427, -0.4027, -0.4468,  0.0340, -0.1474, -0.1838, -0.0961,
          -0.3438, -0.2225,  0.4665]],

        [[-0.0365,  1.5777, -1.1298,  0.8703, -0.3738, -0.8268,  0.9902,
           0.0297, -1.5376,  0.3096],
         [-0.3521, -4.5794, -0.2453,  1.4905, -0.2080, -0.4197,  1.1899,
           0.1261,  0.2413, -0.1793],
         [-0.4027, -0.4468,  0.0340, -0.1474, -0.1838, -0.0961, -0.3438,
          -0.2225,  0.4665, -0.4017]],

        [[ 1.5777, -1.1298,  0.8703, -0.3738, -0.8268,  0.9902,  0.0297,
          -1.5376,  0.3096,  1.2723],
         [-4.5794, -0.2453,  1.4905, -0.2080, -0.4197,  1.1899,  0.1261,
           0.2413, -0.1793,  0.6815],
         [-0.4468,  0.0340, -0.1474, -0.1838, -0.0961, -0.3438, -0.2225,
           0.4665, -0.4017, -0.5560

ipdb>  q


BdbQuit: 

In [ ]:
lstm = nn.LSTM(INPUT_STEP, OUTPUT_STEP, 2).to(device)
lstm = train(lstm)

In [ ]:
lstm = nn.LSTM(INPUT_STEP, OUTPUT_STEP, 2).to(device)
lstm = train(lstm)

In [ ]:
lstm.eval()
lstm_eval = transformed_data.copy()
for i in range(4, p_data.shape[1]-1):
    print(lstm_eval.shape)
    predict, _ = lstm(torch.unsqueeze(torch.FloatTensor(lstm_eval[-2-INPUT_STEP:-2]), 0))                               
    print(predict.shape)
    lstm_eval = np.append(lstm_eval, predict.squeeze(2).detach().numpy(), axis=0)  # append prediction 

In [ ]:
reconstruct(lstm_eval)

### Transformer

We will learn based on this transformed data

In [ ]:
reconstructed_sklearn = pca.inverse_transform(np.random.random((10,3)))

In [ ]:
reconstructed_sklearn.shape, p_learn.shape

In [ ]:
loss = nn.MSELoss()
transformer_model = nn.Transformer(nhead=N_COMPONENTS, num_encoder_layers=32,   # d_model divisible by nhead
                                   d_model=N_COMPONENTS, batch_first=True).to(device)
optimizer = optim.Adam(transformer_model.parameters(), lr=0.0001)

In [ ]:
def train_tf(model):
    model.train()
    loss = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-8)
    for epoch in range(3):  # loop over the dataset multiple times
        for i in tqdm(range(srcs.shape[1]//100)):
            optimizer.zero_grad()
            running_loss = 0.0

            src = srcs[:,i,:].T
            tgt = torch.unsqueeze(tgts[:,i], dim=0)
            outputs = model(src, tgt)
            l = loss(outputs, tgt)
            l.backward()
            optimizer.step()

            # print statistics
            running_loss += l
        print(f'[{epoch + 1}] loss: {running_loss / 2000:}')
    return model  

In [ ]:
transformer_model = train_tf(transformer_model)

## Reconstruct 

In [ ]:
type(p_data)

In [ ]:
transformed_data.shape

In [ ]:
transformer_model.eval()
transformer_eval = transformed_data.copy()
for i in range(4, p_data.shape[1]-1):
    print(transformer_eval.shape)
    predict = transformer_model(torch.unsqueeze(torch.FloatTensor(transformer_eval[-2-INPUT_STEP:-2]), 0),  # INPUT_STEP steps before
                               torch.unsqueeze(torch.FloatTensor(transformer_eval[-1:]), 0))
    print(predict.shape)
    transformer_eval = np.append(transformer_eval, predict[0].detach().numpy(), axis=0)  # append prediction and 

In [ ]:
transformer_eval.shape

In [ ]:
reconstruct(transformer_eval)

todo

10s
10.1s
10.2s

a common algo to reduce dim, pca. 
- look at robustness for prediction further in time.
    - how to get more data simscale. Horizon 8 hours, train 6h, predict 2h. First step 10 mins
        - Increase data slowly, see the limit of ML methods
        - Tradition ML (lstm, transformers) will have problem. How long can we predict using this?
            - Change ML approach maybe to operator inference